# Polynomial Features & Overfitting

Goal: Understand how feature engineering extends linear models to capture nonlinear patterns, and why this power comes with the risk of overfitting.

## The Linearity Constraint

Linear regression fits models of the form:

$$\hat{y} = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_d x_d$$

This is **linear in the features** $x_i$. But what if the true relationship is nonlinear?

### The Key Insight

Linear regression is linear in the **parameters** $w_i$, not necessarily in the original inputs. We can create new features that are nonlinear transformations of the inputs:

$$\phi(x) = [1, x, x^2, x^3, \ldots, x^p]$$

Then fit a linear model on these **polynomial features**:

$$\hat{y} = w_0 + w_1 x + w_2 x^2 + \cdots + w_p x^p$$

This is still a linear model (linear in $w_i$), but it can capture nonlinear patterns in $x$!

## Polynomial Feature Transformation

### Univariate Case

For a single feature $x$, the polynomial transformation of degree $p$ is:

$$\phi: \mathbb{R} \rightarrow \mathbb{R}^{p+1}$$
$$\phi(x) = [1, x, x^2, \ldots, x^p]^T$$

The design matrix becomes:

$$\Phi = \begin{bmatrix} 1 & x_1 & x_1^2 & \cdots & x_1^p \\ 1 & x_2 & x_2^2 & \cdots & x_2^p \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ 1 & x_n & x_n^2 & \cdots & x_n^p \end{bmatrix}$$

### Multivariate Case

For multiple features, we include all polynomial terms up to degree $p$, including **interaction terms**:

For $x = [x_1, x_2]$ and degree 2:
$$\phi(x) = [1, x_1, x_2, x_1^2, x_1 x_2, x_2^2]$$

The number of features grows combinatorially: $\binom{d + p}{p}$ for $d$ original features and degree $p$.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def polynomial_features(x, degree):
    """
    Create polynomial features for univariate input.
    
    Parameters:
        x: array of shape (n,) - input values
        degree: int - maximum polynomial degree
    
    Returns:
        Phi: array of shape (n, degree+1) - design matrix
    """
    n = len(x)
    Phi = np.zeros((n, degree + 1))
    for p in range(degree + 1):
        Phi[:, p] = x ** p
    return Phi

# Example: transform x = [1, 2, 3] with degree 3
x_example = np.array([1, 2, 3])
Phi_example = polynomial_features(x_example, degree=3)

print("Original x:", x_example)
print("\nPolynomial features (degree 3):")
print("  [1, x, x², x³]")
for i, xi in enumerate(x_example):
    print(f"  x={xi}: {Phi_example[i]}")

## Fitting Polynomial Models

Once we have the polynomial features, fitting is identical to linear regression:

$$\mathbf{w} = (\Phi^T\Phi)^{-1}\Phi^T y$$

The prediction for a new point $x^*$ is:

$$\hat{y}^* = \phi(x^*)^T \mathbf{w} = w_0 + w_1 x^* + w_2 (x^*)^2 + \cdots$$

In [ ]:
# Generate synthetic nonlinear data
np.random.seed(42)
n = 30
x = np.linspace(-3, 3, n)
y_true = 0.5 * x**3 - 2 * x**2 + x + 3  # True cubic relationship
y = y_true + np.random.randn(n) * 3  # Add noise

def fit_polynomial(x, y, degree):
    """Fit polynomial regression and return weights."""
    Phi = polynomial_features(x, degree)
    w = np.linalg.inv(Phi.T @ Phi) @ Phi.T @ y
    return w

def predict_polynomial(x, w):
    """Predict using polynomial weights."""
    degree = len(w) - 1
    Phi = polynomial_features(x, degree)
    return Phi @ w

# Fit models of different degrees
x_plot = np.linspace(-3.5, 3.5, 200)
degrees = [1, 3, 9, 15]

fig = go.Figure()

# Plot data
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data',
                         marker=dict(size=10, color='blue')))

# Plot true function
fig.add_trace(go.Scatter(x=x_plot, y=0.5*x_plot**3 - 2*x_plot**2 + x_plot + 3,
                         mode='lines', name='True function',
                         line=dict(color='black', dash='dash', width=2)))

# Plot polynomial fits
colors = ['green', 'orange', 'red', 'purple']
for deg, color in zip(degrees, colors):
    w = fit_polynomial(x, y, deg)
    y_pred = predict_polynomial(x_plot, w)
    fig.add_trace(go.Scatter(x=x_plot, y=y_pred, mode='lines',
                             name=f'Degree {deg}',
                             line=dict(color=color, width=2)))

fig.update_layout(
    title='Polynomial Regression: Effect of Degree',
    xaxis_title='x', yaxis_title='y',
    yaxis=dict(range=[-30, 30]),
    width=800, height=500
)
fig.show()

## Model Capacity and Flexibility

### What is Capacity?

**Model capacity** (or complexity) refers to the variety of functions a model can fit. Higher capacity means the model can represent more complex patterns.

For polynomial regression:
- **Degree 1** (linear): Can only fit straight lines
- **Degree 2** (quadratic): Can fit parabolas
- **Degree $n-1$**: Can pass through all $n$ data points exactly!

### Degrees of Freedom

A degree-$p$ polynomial has $p+1$ parameters (degrees of freedom). With $n$ data points:

- If $p + 1 < n$: System is **overdetermined** (more equations than unknowns)
- If $p + 1 = n$: System is **exactly determined** (unique solution through all points)
- If $p + 1 > n$: System is **underdetermined** (infinitely many solutions)

### The Interpolation Trap

When $p \geq n - 1$, we can achieve **zero training error**! The polynomial passes through every data point. But is this good?

In [ ]:
# Demonstrate the interpolation trap
np.random.seed(42)
n_small = 10
x_small = np.linspace(-2, 2, n_small)
y_small = np.sin(x_small) + np.random.randn(n_small) * 0.3

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data Fit', 'Training MSE vs Degree'))

# Left plot: visualize fits
x_dense = np.linspace(-2.5, 2.5, 200)
for deg in [1, 3, 5, 9]:
    w = fit_polynomial(x_small, y_small, deg)
    y_pred = predict_polynomial(x_dense, w)
    # Clip extreme values for visualization
    y_pred = np.clip(y_pred, -5, 5)
    fig.add_trace(go.Scatter(x=x_dense, y=y_pred, mode='lines', name=f'Degree {deg}'),
                  row=1, col=1)

fig.add_trace(go.Scatter(x=x_small, y=y_small, mode='markers', name='Data',
                         marker=dict(size=12, color='black')), row=1, col=1)

# Right plot: training error vs degree
degrees = range(1, n_small)
train_errors = []
for deg in degrees:
    w = fit_polynomial(x_small, y_small, deg)
    y_pred = predict_polynomial(x_small, w)
    mse = np.mean((y_small - y_pred)**2)
    train_errors.append(mse)

fig.add_trace(go.Scatter(x=list(degrees), y=train_errors, mode='lines+markers',
                         name='Training MSE', marker=dict(size=10)),
              row=1, col=2)

fig.update_yaxes(range=[-3, 3], row=1, col=1)
fig.update_xaxes(title_text='x', row=1, col=1)
fig.update_yaxes(title_text='y', row=1, col=1)
fig.update_xaxes(title_text='Polynomial Degree', row=1, col=2)
fig.update_yaxes(title_text='MSE', type='log', row=1, col=2)

fig.update_layout(width=1000, height=400, title='Model Capacity: More Degrees = Lower Training Error')
fig.show()

print(f"With n={n_small} points, degree {n_small-1} achieves near-zero training error.")
print(f"Training MSE at degree {n_small-1}: {train_errors[-1]:.2e}")

## Overfitting: When Models Memorize Instead of Learn

### Definition

**Overfitting** occurs when a model learns patterns specific to the training data that don't generalize to new data. The model memorizes noise rather than learning the underlying relationship.

### Symptoms of Overfitting

1. **Low training error, high test error**: The gap between them is the key signal
2. **Large weight magnitudes**: Overfitted polynomials have huge coefficients
3. **Oscillating predictions**: Wild swings between data points
4. **Sensitivity to small changes**: Adding one point dramatically changes the fit

### Why Does Overfitting Happen?

Consider fitting a degree-9 polynomial to 10 noisy points:
- The model has 10 parameters and 10 data points
- It can achieve perfect fit by solving a system of 10 equations with 10 unknowns
- But it has learned the **noise**, not just the **signal**

### The Mathematical Perspective

Observed data: $y = f(x) + \epsilon$ where $f$ is the true function and $\epsilon$ is noise.

An overfitted model $\hat{f}$ satisfies:
- $\hat{f}(x_i) \approx y_i = f(x_i) + \epsilon_i$ (fits training points)
- But $\hat{f}(x^*) \neq f(x^*)$ for new points $x^*$ (poor generalization)

The model has captured $\epsilon_i$ (noise) in addition to $f(x_i)$ (signal).

In [ ]:
# Demonstrate overfitting with train/test split
np.random.seed(42)

# Generate more data
n_total = 50
x_all = np.random.uniform(-3, 3, n_total)
y_true_all = np.sin(x_all) * 2 + 0.5 * x_all  # True underlying function
y_all = y_true_all + np.random.randn(n_total) * 0.5  # Add noise

# Split into train and test
n_train = 30
indices = np.random.permutation(n_total)
train_idx, test_idx = indices[:n_train], indices[n_train:]

x_train, y_train = x_all[train_idx], y_all[train_idx]
x_test, y_test = x_all[test_idx], y_all[test_idx]

# Evaluate different degrees
degrees = range(1, 20)
train_mse = []
test_mse = []

for deg in degrees:
    w = fit_polynomial(x_train, y_train, deg)
    
    # Training error
    y_train_pred = predict_polynomial(x_train, w)
    train_mse.append(np.mean((y_train - y_train_pred)**2))
    
    # Test error
    y_test_pred = predict_polynomial(x_test, w)
    test_mse.append(np.mean((y_test - y_test_pred)**2))

# Plot the classic overfitting curve
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(degrees), y=train_mse, mode='lines+markers',
                         name='Training Error', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=list(degrees), y=test_mse, mode='lines+markers',
                         name='Test Error', line=dict(color='red')))

# Find optimal degree
best_deg = degrees[np.argmin(test_mse)]
fig.add_vline(x=best_deg, line_dash='dash', line_color='green',
              annotation_text=f'Best: degree {best_deg}')

fig.update_layout(
    title='The Overfitting Curve: Training vs Test Error',
    xaxis_title='Polynomial Degree',
    yaxis_title='Mean Squared Error',
    yaxis_type='log',
    width=800, height=500
)
fig.show()

print(f"Best test performance at degree {best_deg}")
print(f"  Training MSE: {train_mse[best_deg-1]:.4f}")
print(f"  Test MSE: {test_mse[best_deg-1]:.4f}")

## Underfitting: When Models Are Too Simple

The opposite of overfitting is **underfitting**: the model is too simple to capture the underlying pattern.

### Symptoms of Underfitting

1. **High training error AND high test error**: Both are bad
2. **Systematic patterns in residuals**: The model misses structure
3. **Simple, smooth predictions**: May look "reasonable" but miss complexity

### The Three Regimes

| Regime | Training Error | Test Error | Diagnosis |
|--------|---------------|------------|------------|
| Underfitting | High | High | Model too simple |
| Good fit | Low | Low | Model complexity appropriate |
| Overfitting | Very low | High | Model too complex |

In [ ]:
# Visualize the three regimes
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=('Underfitting (Degree 1)', 
                                   f'Good Fit (Degree {best_deg})', 
                                   'Overfitting (Degree 15)'))

x_dense = np.linspace(-3.5, 3.5, 200)
regimes = [1, best_deg, 15]

for i, deg in enumerate(regimes, 1):
    w = fit_polynomial(x_train, y_train, deg)
    y_pred_dense = predict_polynomial(x_dense, w)
    y_pred_dense = np.clip(y_pred_dense, -10, 10)  # Clip for visualization
    
    # Plot prediction
    fig.add_trace(go.Scatter(x=x_dense, y=y_pred_dense, mode='lines',
                             name=f'Degree {deg}', line=dict(color='red', width=2)),
                  row=1, col=i)
    
    # Plot training data
    fig.add_trace(go.Scatter(x=x_train, y=y_train, mode='markers',
                             name='Train', marker=dict(size=8, color='blue')),
                  row=1, col=i)
    
    # Plot test data
    fig.add_trace(go.Scatter(x=x_test, y=y_test, mode='markers',
                             name='Test', marker=dict(size=8, color='green', symbol='x')),
                  row=1, col=i)
    
    # Compute errors
    train_err = np.mean((y_train - predict_polynomial(x_train, w))**2)
    test_err = np.mean((y_test - predict_polynomial(x_test, w))**2)
    
    fig.add_annotation(x=0, y=-8, text=f'Train: {train_err:.3f}<br>Test: {test_err:.3f}',
                       showarrow=False, row=1, col=i)

fig.update_yaxes(range=[-10, 10])
fig.update_layout(width=1200, height=400, showlegend=False,
                  title='Three Regimes: Underfitting, Good Fit, Overfitting')
fig.show()

## Weight Magnitudes: A Window into Overfitting

Overfitted models often have **extremely large weights**. This is because:

1. To pass through noisy points exactly, the polynomial must oscillate rapidly
2. Rapid oscillations require large positive and negative coefficients that nearly cancel
3. Small changes in input cause large changes in output

### The Polynomial Coefficient Explosion

For a degree-$p$ polynomial with coefficients $w_0, w_1, \ldots, w_p$:

$$\hat{y} = w_0 + w_1 x + w_2 x^2 + \cdots + w_p x^p$$

If $|w_i|$ are large and alternating in sign, the polynomial will oscillate wildly. This is a telltale sign of overfitting.

In [ ]:
# Analyze weight magnitudes for different degrees
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Weight Magnitudes by Degree', 'Max |Weight| vs Degree'))

degrees_to_show = [3, 7, 12, 17]
colors = ['green', 'blue', 'orange', 'red']

max_weights = []
all_degrees = range(1, 20)

for deg in all_degrees:
    w = fit_polynomial(x_train, y_train, deg)
    max_weights.append(np.max(np.abs(w)))

# Left plot: weight values for selected degrees
for deg, color in zip(degrees_to_show, colors):
    w = fit_polynomial(x_train, y_train, deg)
    fig.add_trace(go.Bar(x=[f'w{i}' for i in range(len(w))], y=w,
                         name=f'Degree {deg}', marker_color=color, opacity=0.7),
                  row=1, col=1)

# Right plot: max weight vs degree
fig.add_trace(go.Scatter(x=list(all_degrees), y=max_weights, mode='lines+markers',
                         name='Max |weight|', line=dict(color='purple')),
              row=1, col=2)

fig.update_yaxes(title_text='Weight Value', row=1, col=1)
fig.update_yaxes(title_text='Max |Weight|', type='log', row=1, col=2)
fig.update_xaxes(title_text='Coefficient', row=1, col=1)
fig.update_xaxes(title_text='Polynomial Degree', row=1, col=2)

fig.update_layout(width=1000, height=400, 
                  title='Weight Explosion: A Signature of Overfitting')
fig.show()

print("\nWeight magnitudes grow exponentially with degree!")
print(f"Degree 3: max|w| = {max_weights[2]:.2f}")
print(f"Degree 10: max|w| = {max_weights[9]:.2e}")
print(f"Degree 15: max|w| = {max_weights[14]:.2e}")

## Numerical Stability: The Vandermonde Matrix Problem

High-degree polynomial regression has serious numerical issues.

### The Vandermonde Matrix

The polynomial design matrix $\Phi$ is called a **Vandermonde matrix**:

$$\Phi = \begin{bmatrix} 1 & x_1 & x_1^2 & \cdots & x_1^p \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ 1 & x_n & x_n^2 & \cdots & x_n^p \end{bmatrix}$$

### Why It's Ill-Conditioned

1. **Columns become similar**: $x^{p-1}$ and $x^p$ are nearly collinear for large $p$
2. **Huge value range**: If $x \in [-3, 3]$, then $x^{15}$ ranges from $-14$ million to $+14$ million!
3. **Poor condition number**: $\kappa(\Phi^T\Phi)$ grows exponentially with degree

### Consequences

- $(\Phi^T\Phi)^{-1}$ becomes numerically unstable
- Small changes in data cause huge changes in weights
- Floating-point errors get amplified

**Practical fix**: Use orthogonal polynomials (Legendre, Chebyshev) or regularization.

In [ ]:
# Demonstrate the ill-conditioning of Vandermonde matrices
degrees = range(1, 20)
condition_numbers = []

for deg in degrees:
    Phi = polynomial_features(x_train, deg)
    # Condition number of Phi^T Phi
    cond = np.linalg.cond(Phi.T @ Phi)
    condition_numbers.append(cond)

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(degrees), y=condition_numbers, mode='lines+markers',
                         line=dict(color='red', width=2)))

# Add reference line for machine epsilon
machine_eps_inv = 1 / np.finfo(float).eps
fig.add_hline(y=machine_eps_inv, line_dash='dash', line_color='gray',
              annotation_text='~Machine precision limit')

fig.update_layout(
    title='Condition Number of Φ^TΦ vs Polynomial Degree',
    xaxis_title='Polynomial Degree',
    yaxis_title='Condition Number',
    yaxis_type='log',
    width=800, height=500
)
fig.show()

print("Condition numbers grow exponentially:")
for deg in [5, 10, 15]:
    print(f"  Degree {deg}: κ = {condition_numbers[deg-1]:.2e}")
print(f"\nMachine epsilon: {np.finfo(float).eps:.2e}")
print(f"When κ > 1/eps ≈ {machine_eps_inv:.2e}, results become unreliable.")

## Generalization: The Ultimate Goal

### What We Really Want

The goal of machine learning is not to fit training data well, but to **generalize** to unseen data.

**Generalization error** (or risk) is the expected error on new data:

$$R(\hat{f}) = \mathbb{E}_{(x,y) \sim P}[L(y, \hat{f}(x))]$$

where $P$ is the true data distribution and $L$ is our loss function.

### The Generalization Gap

$$\text{Generalization Gap} = R(\hat{f}) - \hat{R}_{\text{train}}(\hat{f})$$

where $\hat{R}_{\text{train}}$ is the training error.

- **Small gap**: Model generalizes well
- **Large gap**: Model overfits

### Connection to Bias-Variance (Preview)

The generalization error can be decomposed (covered in Section 6):

$$\text{Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Noise}$$

- High-degree polynomials: Low bias, high variance (overfitting)
- Low-degree polynomials: High bias, low variance (underfitting)

In [ ]:
# Demonstrate variance in predictions: fit same model to different data samples
np.random.seed(0)

def generate_data(n, noise=0.5):
    x = np.random.uniform(-3, 3, n)
    y_true = np.sin(x) * 2 + 0.5 * x
    y = y_true + np.random.randn(n) * noise
    return x, y

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Low Complexity (Degree 2): Low Variance',
                                   'High Complexity (Degree 12): High Variance'))

x_plot = np.linspace(-3.5, 3.5, 200)
n_samples = 10
degrees_compare = [2, 12]

for col, deg in enumerate(degrees_compare, 1):
    all_predictions = []
    
    for i in range(n_samples):
        x_sample, y_sample = generate_data(30)
        w = fit_polynomial(x_sample, y_sample, deg)
        y_pred = predict_polynomial(x_plot, w)
        y_pred = np.clip(y_pred, -8, 8)
        all_predictions.append(y_pred)
        
        fig.add_trace(go.Scatter(x=x_plot, y=y_pred, mode='lines',
                                 line=dict(color='blue', width=1), opacity=0.3,
                                 showlegend=False),
                      row=1, col=col)
    
    # Plot mean prediction
    mean_pred = np.mean(all_predictions, axis=0)
    fig.add_trace(go.Scatter(x=x_plot, y=mean_pred, mode='lines',
                             line=dict(color='red', width=3),
                             name='Mean prediction'),
                  row=1, col=col)
    
    # Plot true function
    y_true_plot = np.sin(x_plot) * 2 + 0.5 * x_plot
    fig.add_trace(go.Scatter(x=x_plot, y=y_true_plot, mode='lines',
                             line=dict(color='black', width=2, dash='dash'),
                             name='True function'),
                  row=1, col=col)

fig.update_yaxes(range=[-8, 8])
fig.update_layout(width=1000, height=450,
                  title='Variance in Predictions: Same Model, Different Training Sets')
fig.show()

print("Each blue line is the same model trained on a different random sample.")
print("High-complexity models vary wildly depending on the training data!")

## Practical Considerations

### When to Use Polynomial Features

**Good candidates:**
- Physical processes with known polynomial relationships
- Interaction effects between features ($x_1 \cdot x_2$)
- Low-degree polynomials (2-4) for slight nonlinearity

**Be careful with:**
- High-degree polynomials (extrapolation is dangerous!)
- Many features (combinatorial explosion of terms)
- Unscaled data (numerical instability)

### Alternatives to High-Degree Polynomials

1. **Regularization** (Ridge/Lasso): Control weight magnitudes (Section 7)
2. **Splines**: Piecewise polynomials with local support
3. **Kernel methods**: Implicit high-dimensional features (SVMs)
4. **Tree-based models**: Piecewise constant approximations
5. **Neural networks**: Learned feature representations

### Feature Scaling for Polynomials

**Always scale features before creating polynomials!**

If $x \in [0, 1000]$, then $x^3 \in [0, 10^9]$. This causes:
- Numerical overflow
- Ill-conditioned matrices
- Gradient descent convergence issues

Standard approach: Standardize first, then create polynomial features.

In [ ]:
# Demonstrate the importance of scaling
np.random.seed(42)

# Generate data with large range
x_large = np.random.uniform(0, 100, 30)
y_large = 0.001 * x_large**2 - 0.1 * x_large + 5 + np.random.randn(30) * 2

# Without scaling
Phi_unscaled = polynomial_features(x_large, degree=5)
cond_unscaled = np.linalg.cond(Phi_unscaled.T @ Phi_unscaled)

# With scaling (standardization)
x_scaled = (x_large - x_large.mean()) / x_large.std()
Phi_scaled = polynomial_features(x_scaled, degree=5)
cond_scaled = np.linalg.cond(Phi_scaled.T @ Phi_scaled)

print("Effect of Scaling on Numerical Stability")
print("=" * 45)
print(f"Original x range: [{x_large.min():.1f}, {x_large.max():.1f}]")
print(f"Scaled x range:   [{x_scaled.min():.2f}, {x_scaled.max():.2f}]")
print(f"\nCondition number (degree 5):")
print(f"  Unscaled: {cond_unscaled:.2e}")
print(f"  Scaled:   {cond_scaled:.2e}")
print(f"  Improvement: {cond_unscaled/cond_scaled:.0f}x better!")

## Summary

### Key Takeaways

1. **Polynomial features** extend linear models to capture nonlinear patterns
   - Transform: $x \rightarrow [1, x, x^2, \ldots, x^p]$
   - Still linear in parameters, nonlinear in inputs

2. **Model capacity** increases with polynomial degree
   - More parameters = more flexibility
   - Degree $n-1$ can interpolate $n$ points exactly

3. **Overfitting** occurs when capacity exceeds what data supports
   - Symptoms: Low training error, high test error, large weights
   - Model learns noise, not just signal

4. **Underfitting** occurs when capacity is insufficient
   - Symptoms: High training AND test error
   - Model misses important patterns

5. **Numerical issues** plague high-degree polynomials
   - Vandermonde matrices are ill-conditioned
   - Always scale features before polynomial expansion

### Connections to Other Topics

- **Section 4** (Train/Val/Test): How to properly evaluate generalization
- **Section 5** (Feature Scaling): Why and how to scale before polynomial features
- **Section 6** (Bias-Variance): Formal framework for the overfitting tradeoff
- **Section 7** (Regularization): How to control overfitting mathematically